In [1]:
from helper import *
from skimage.feature import local_binary_pattern

In [2]:
SUBSET = '200k'
DATA_DIR = 'data/'
DATA_FILENAME = '1d_subset' + SUBSET

In [3]:
(X_train, X2_train, y_train), (X_test, X2_test, y_test) = \
    load2x_npz(DATA_DIR + DATA_FILENAME + '.npz')

((200000, 9216), dtype('uint8')) ((200000, 2304), dtype('uint8')) ((200000,), dtype('uint8'))
((20000, 9216), dtype('uint8')) ((20000, 2304), dtype('uint8')) ((20000,), dtype('uint8'))


In [4]:
# Local Binary Pattern Histogram
def lbpv_histogram(img, b, n):
    patterns = local_binary_pattern(img, 8, 1, 'var')
    hist, _ = np.histogram(patterns, bins=np.arange(0, b*n + 1, b))
    return hist.astype('int16')

In [5]:
# Local Binary Pattern Histogram
def lbpu_histogram(img):
    patterns = local_binary_pattern(img, 8, 1, 'uniform')
    hist, _ = np.histogram(patterns, bins=np.arange(10 + 1))
    return hist.astype('int16')

In [6]:
# Turn a <v> of size 9216 (96x96) + 2304 (48x48) into 1-D LDP
# feature vector of size 40 (10x2x2) (2x2LDP histogram)
def vec_to_feat(v):
    img1 = v[:9216].reshape(96, 96)
    img2 = v[9216:].reshape(48, 48)
    lbp1 = lbpu_histogram(img1)
    lbp2 = lbpu_histogram(img2)
    lbp3 = lbpv_histogram(img1, 400, 10)
    lbp4 = lbpv_histogram(img2, 300, 10)
    return np.concatenate((lbp1, lbp2, lbp3, lbp4))

In [7]:
Xf_train = np.apply_along_axis(delayed(vec_to_feat), 1, np.hstack((X_train, X2_train)))
Xf_train = np.concatenate(compute([x for x in Xf_train]))
Xf_train.shape, Xf_train.dtype

((200000, 40), dtype('int16'))

In [8]:
Xf_test = np.apply_along_axis(delayed(vec_to_feat), 1, np.hstack((X_test, X2_test)))
Xf_test = np.concatenate(compute([x for x in Xf_test]))
Xf_test.shape, Xf_test.dtype

((20000, 40), dtype('int16'))

In [9]:
SUBSET_FILENAME = '2x2lbp_subset' + SUBSET
np.savez(DATA_DIR + SUBSET_FILENAME,
         X_train=Xf_train, y_train=y_train,
         X_test=Xf_test, y_test=y_test)

In [10]:
GBT(Xf_train, Xf_test, y_train, y_test)

n_est: Train, Test
400  : 83.0,  82.4
600  : 83.7,  82.8
800  : 84.2,  83.1
